### Домашнее задание к лекции «Понятие класса»

#### Задание 1.
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса 
с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [50]:
import requests

In [51]:
url_daily = 'https://www.cbr-xml-daily.ru/daily_json.js'
def max_course(url):
    full_data = requests.get(url).json()
    valute_data = {}
    for params in full_data['Valute'].values():
        valute_data[params['Name']] = params['Value']/params['Nominal']
    max_valute_name = max(valute_data, key=lambda key: valute_data[key])
    return f'Валюта с максимальным курсом: {max_valute_name} - {valute_data[max_valute_name]}'

In [52]:
max_course(url_daily)

'Валюта с максимальным курсом: СДР (специальные права заимствования) - 112.7784'

#### Задание 2.
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [65]:
class Rate:
    def __init__(self, format_='value', diff=True): #Добавляю параметр diff (по-умолчанию True)
        self.format = format_
        if format_ == 'value':
            self.diff = diff #Присваиваю значение параметра diff переменной diff, только если format_ == 'value'
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if self.diff == True: # проверяю значение diff, только если format == 'value'
                    return response[currency]['Previous'] - response[currency]['Value']
                    # если diff == True, вывожу разницу курсов
                else:
                    return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [82]:
d = Rate(format_='value')
d.usd()

0.5742999999999938

In [83]:
c = Rate(format_='value', diff = False)
c.eur()

93.3447

In [84]:
f = Rate(format_='full')
f.brl()

{'ID': 'R01115',
 'NumCode': '986',
 'CharCode': 'BRL',
 'Nominal': 1,
 'Name': 'Бразильский реал',
 'Value': 13.9301,
 'Previous': 14.0304}

#### Задание 3.
Напишите класс Designer, который учитывает количество международных премий. 
Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла. Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

In [174]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [208]:
class Designer(Employee):
    def __init__(self, name, seniority, awards = 2):
        super().__init__(name, seniority)
        self.awards = awards
    
    def awards_up(self):
        """Добавляю метод для получение международной премии"""
        self.awards += 1
    
    def publish_grade(self):
        """Переопределяю метод publish_grade, чтобы можно было вывести кол-во премий """
        print(self.name, self.grade, self.awards)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        self.seniority += 1
        
        # условие повышения сотрудника
        if (self.seniority + self.awards*2) % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [209]:
elena = Designer('Elena', 0, 1)

In [210]:
elena.publish_grade()

Elena 1 1


In [211]:
elena.awards_up()
elena.publish_grade()

Elena 1 2


In [212]:
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Elena 1 2
Elena 1 2
Elena 2 2
Elena 2 2
Elena 2 2
Elena 2 2
Elena 2 2
Elena 2 2
Elena 2 2
Elena 3 2
Elena 3 2
Elena 3 2
Elena 3 2
Elena 3 2
Elena 3 2
Elena 3 2
Elena 4 2
Elena 4 2
Elena 4 2
Elena 4 2
